<div style="text-align: center;"><font size="5"><strong>LIBRARY</strong></font></div>

+ import time: Set the sleep time of each scroll when the program send the key down to site.
+ import requests: Request the web page.
+ import pandas as pd: Work with csv.
+ import urllib.robotparser: To know can the program crawl the web page.
+ from bs4 import BeautifulSoup: Work with html, find element and the value.
+ from selenium import webdriver: Because Soundclound blocked with javascipt so that we have to use this library to call chrome driver to get the page source after javascript loaded.
+ from selenium.webdriver.common.keys import Keys: Send the key to the website.

In [ ]:
import time
import requests
import pandas as pd
import urllib.robotparser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

<div style="text-align: center;"><font size="5"><strong>GLOBAL VARIABLE</strong></font></div>

+ limit: The quantity of users.
+ uniq_user: The list created to store set of unique users.

In [ ]:
uniq_user=[] #unique user list
limit=1001

<div style="text-align: center;"><font size="5"><strong>CHECK ROBOTS.TXT FILE</strong></font></div>

+ This function to check the robots text file follow this link: https://soundcloud.com/discover/

In [ ]:
rp = urllib.robotparser.RobotFileParser(url='https://soundcloud.com/robots.txt')
rp.read()
rp.can_fetch('*','https://soundcloud.com/discover/' )

<div style="text-align: center;"><font size="5"><strong>FUNCTIONS</strong></font></div>

+ <font size="3">We used a combined HTML parsing method using the selenium library to get the data from website soundcloud.com.
Here are some functions used in the program:</font>

+ page_scroll_down(driver, number_of_key_down=8):This function help us use selenium to scroll specified web page with two argument driver (driver of link to specified web page) and number_of_key_down(number of page scrolls).

In [ ]:
def page_scroll_down(driver, number_of_key_down=8):
    page_body = driver.find_element_by_tag_name('body')
    num_of_pagedowns = number_of_key_down
    while num_of_pagedowns:
        page_body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_of_pagedowns-=1

+ clone_and_save_url_to_playlist(): This function help us filter and save links to top playlist. We have filtered the featured playlists in the link 'https://soundcloud.com/discover/', and saved their links to a csv file.

In [ ]:
def clone_and_save_url_to_playlist():
    url = 'https://soundcloud.com/discover/'
    driver = webdriver.Chrome('chromedriver.exe')
    driver.get(url)
    time.sleep(1)
    
    driver.find_element_by_xpath("//button[@id='onetrust-accept-btn-handler']").click()
    
    for i in range(1, 9):
        for j in range(10):
            driver.find_element_by_xpath(f'//*[@id="content"]/div/div/div[1]/div[3]/div/ul/li[{i}]/div/div[2]/div[2]/button').click()
    
    html = driver.page_source
    soup = BeautifulSoup(html)
    url_to_playlist = soup.find_all('a',{'class':'playableTile__artworkLink'})
    # save url to playlist to text file
    with open('sc_url_to_playlist.txt','w') as writer:
        for href in url_to_playlist:
            writer.write('https://soundcloud.com' + href['href'] + '\n')
    driver.close()

+ get_url_to_playlist():This function help us get the link from a csv file with one argument is file_name (name of the file we wanna read the data frame).

In [ ]:
def get_url_to_playlist(file_name='sc_url_to_playlist.txt'):
    urls = []
    # get url
    with open(file_name) as reader:
        for line in reader:
            urls.append(line.rstrip())
    return urls

+ crawl_track_info(url_to_profile, user_name, driver, writer):This function help us crawl track info from each user's track page and save info to csv file 
    We go to the track page's link, then scroll to the bottom of the page to get all the links to each track.
    After that, we and each track to get information such as name, author, views, likes, link to track,....
    If after filtering, but the necessary information is not available, it will return None.

In [ ]:
def crawl_track_info(url_to_profile, user_name, driver, writer):
    
    tracks_tab_url = url_to_profile + '/tracks'
    driver.get(tracks_tab_url)
    page_scroll_down(driver, 3)
    html = driver.page_source
    soup = BeautifulSoup(html)
    track_url = soup.find_all('a',{'class':'soundTitle__title'})
        
    if len(track_url) == 0: writer.write(f'None>>{user_name}>>None>>None>>None>>>None>>>None>>>None\n')
    else:
        for href in track_url:
            url_track_info = 'https://soundcloud.com' + href['href'].strip()
            driver.get(url_track_info)
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html)
            
            track_info = ''
            try: track_info += (soup.find('div', {'class':'soundTitle__titleHeroContainer'})).find('h1',{'class':'soundTitle__title'}).text.replace('\n', '') + '>>'
            except: track_info += 'None>>'
            track_info += user_name + '>>'
            try: track_info += soup.find('a',{'class':'sc-tag-large'}).text.replace('\n', '') + '>>'
            except: track_info += 'None>>'
            track_info += f'{url_track_info}>>'
            try: track_info += (soup.find('span',{'class':'sc-ministats sc-ministats-medium sc-ministats-plays sc-text-secondary'})).find('span',{'class': 'sc-visuallyhidden'}).text.split(' ')[0] + '>>'
            except: track_info += 'None>>'
            try: track_info += soup.find('a',{'class':'sc-ministats sc-ministats-medium sc-ministats-likes sc-link-secondary'}).text.replace('View all likes', '').replace('\n', '') + '>>'
            except: track_info += 'None>>'
            try: track_info += soup.find('a',{'class':'sc-ministats sc-ministats-medium sc-ministats-reposts sc-link-secondary'}).text.replace('View all reposts', '').replace('\n', '')
            except: track_info += 'None'
                
            writer.write(track_info + '\n')

+ crawl_playlist_info(url_to_profile, user_name, driver, writer): crawl playlist info from each user's playlist page and save info to csv file 
    We go to the playlist page's link, then scroll to the bottom of the page to get all the links to each playlist.
    After that, we and each playlist to get information such as name, author, views, likes, link to playlist,....
    If after filtering, but the necessary information is not available, it will return None.

In [ ]:
def crawl_playlist_info(url_to_profile, user_name, driver, writer):
    playlist_tab_url = url_to_profile + '/sets'
    driver.get(playlist_tab_url)
    page_scroll_down(driver, 3)
    html = driver.page_source
    soup = BeautifulSoup(html)
    playlist_url = soup.find_all('a',{'class':'sound__coverArt'})

    if len(playlist_url) == 0:
        writer.write(f'None>>None>>{user_name}>>None>>None>>None\n')
    else:
        for href in playlist_url:
            url_playlist_info = 'https://soundcloud.com' + href['href'].strip()
            driver.get(url_playlist_info)
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html)
            
            playlist_info = ''
            try: playlist_info += (soup.find('div', {'class':'soundTitle__titleHeroContainer'})).find('h1',{'class':'soundTitle__title'}).text.replace('\n', '') + '>>'
            except: playlist_info += 'None>>'
            try: playlist_info += soup.find('a',{'class':'sc-tag-large'}).text.replace('\n', '') + '>>'
            except: playlist_info += 'None>>'
            try: playlist_info += f'{user_name}>>'
            except: playlist_info += 'None>>'
            try: playlist_info += f'{url_playlist_info}>>'
            except: playlist_info += 'None>>'
            try: playlist_info += soup.find('div',{'class': 'genericTrackCount__title sc-text-h1'}).text.strip() + '>>'
            except: playlist_info += 'None>>'
            try: playlist_info += (soup.find('ul',{'class':'sc-ministats-group-right'})).find('a',{'class':'sc-ministats-likes'}).text.replace('\n', '').replace('View all likes','') + '>>' #like
            except: playlist_info += 'None>>'
            try: playlist_info += (soup.find('ul',{'class':'sc-ministats-group-right'})).find('a',{'class':'sc-ministats-reposts'}).text.replace('\n', '').replace('View all reposts','') #repost
            except: playlist_info += 'None'
                
            writer.write(f'{playlist_info}\n')

+ HTML_Parser():get data like user info, track info and playlist info
    We get link featured playlists and filter users with hit tracks.
    Then, go to each user's profile to get info and links to their tracks and playlists.
    Then we get track info and playlist info using other functions.
    After that, we will save profile info, track info and playlist info to the csv file.

In [ ]:
def HTML_Parser():
    urls = get_url_to_playlist()
    driver = webdriver.Chrome('chromedriver.exe')
    with open('user.csv', 'w', encoding='utf-8') as writer1:
        with open('track_list.csv', 'w', encoding='utf-8') as writer2:
            with open('play_list.csv', 'w', encoding='utf-8') as writer3:
                count=0
                writer1.write('user_name>>follower>>following>>tracks\n')
                writer2.write('name of track>>user_name>>genre>>link to track>>playbacks>>likes of track>>reposts of track\n')
                writer3.write('name of playlist>>genre>>writer>>link to playlist>>track count>>likes of playlist>>reposts of playlist\n')
                for url in urls:
                    
                    # Go to user profile and crawl user info
                    driver.get(url)
                    page_scroll_down(driver, 8)
                    html = driver.page_source
                    soup = BeautifulSoup(html)
                    list_href_of_user_profile = soup.find_all('a',{'class':'trackItem__username'})
                    if len(list_href_of_user_profile) == 0:
                        print('Network error at this session please start again or wait for the next!')
                    else:
                        for href in list_href_of_user_profile:
                            if href not in uniq_user:
                                uniq_user.append(href)
                                url_to_profile = 'https://soundcloud.com' +  href['href']
                                driver.get(url_to_profile)
                                time.sleep(2)
                                html = driver.page_source
                                soup = BeautifulSoup(html)
                                user_name = ''
                                try: user_name = (soup.find('div',{'class':'profileHeaderInfo__content'})).find('h2',{'class': 'profileHeaderInfo__userName'}).text.replace('\n', '')[4:-4]
                                except: user_name = 'None'
                                user_profile = user_name + '>>'
                                #============================================
                                user_stats = soup.find_all('div',{'class':'infoStats__value'})
                                if len(user_stats) == 0: user_profile += 'None>>None>>None>>'
                                else:
                                    for stat in user_stats: user_profile += stat.text.replace('\n', '') + '>>'
                                writer1.write(user_profile[:-2] + '\n')    
                            # Go to track tab and crawl track info
                                crawl_track_info(url_to_profile, user_name, driver, writer2)
                            #===========================================
                            # Go to playlist tab and crawl playlist info
                                crawl_playlist_info(url_to_profile, user_name, driver, writer3)
                            #===========================================
                                count+=1
                                if count==limit:return
    driver.close()

+ SC_read_data(): This function help us load the data frame from a csv file with two argument is file_name (name of the file we wanna read the data frame) and sep (the separate for every element of csv file). We use pandas to read csv by read_csv() function, load the file_name and sep into that and store the data frame into variable named data and return the data. In the data frame of the output we have three file of data is:
    + user.csv
    + track_list.csv
    + play_list.csv

+ NOTICE:
    + The separate default is >> because maybe the string is included space or ; so we use >> not sure for 100% but it can get 90% of the result.
         

In [ ]:
def SC_read_data(file_name='track_list.csv', sep='>>'):
    data = pd.read_csv(file_name, sep=sep)
    return data
            

<div style="text-align: center;"><font size="5"><strong>MAIN</strong></font></div>

+ Lines below are used as the main function to run the program from A-Z.

+ NOTICE:
    + So sory if the data get the name of track is none: During the progress it maybe the weak network affect that.
    + Time to run < 1000 record is so long please consider before the progress or you can change the limit variable on the Variable cell to limit the test case.
    + Please choose Kernel ==> Restart & Run All.

# Parse the web page

In [ ]:
# clone_and_save_url_to_playlist()
HTML_Parser()

# Ten first line of the data frame user

In [ ]:
df = SC_read_data('user.csv')
df[:10]

# Info of the data frame user

In [ ]:
df.info()

# Ten first line of the data frame tracklist

In [ ]:
df = SC_read_data('track_list.csv')
df[:10]

# Info of the data frame tracklist

In [ ]:
df.info()

# Ten first line of the data frame playlist

In [ ]:
df = SC_read_data('play_list.csv')
df[:10]

# Info of the data frame playlist

In [ ]:
df.info()